# TP : RL OpenAI Gym

## Objectifs du TP

- Découvrir les propriétés de l'environnement du problème du Taxi sur Gym
- Implémenter l’algorithme du Q-Learning
- Essayer différents paramètres pour l’équation de Bellman
- Exécuter l’algorithme du Q-Learning sur un autre environnement : MountainCar

## Python et importation des bibliothèques utilisées pour le TP

Pour ce TP, Python 3 est utilisé ainsi que plusieurs bibliothèques qu'il faut charger :
1. **gym** : permet de développer et de comparer des algorithmes d'apprentissage par renforcement.
- Documentation : https://gym.openai.com/docs/
- Tous les environnements : https://gym.openai.com/envs/#classic_control
1. **numpy** : permet de manipuler efficacement des grandes matrices  
http://www.numpy.org
1. **matplotlib** : permet de tracer et visualiser des données sous formes de graphiques
https://matplotlib.org

### Si vous utilisez un ordinateur du département, lancez le notebook avec Jupyter (commande : jupyter notebook) et installez les modules suivants.

In [ ]:
!pip install gym
!pip install matplotlib
!pip install pyglet

### **Important** : Pour éviter de faire planter le notebook, ne pas oublier de faire un env.close() à la fin de chaque cellule où vous affichez l'environnement avec env.render()</font>

In [ ]:
# Import des modules nécessaires
import gym
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output

## **A. Taxi**

Commencons par reprendre l'exemple du Taxi vu en cours pour découvrir Gym et implémenter l'algorithme du Q-Learning.

A titre d'information, voici le code source de l'environnement : https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py

In [ ]:
# Configuration de l'environnement
env = gym.make('Taxi-v3')

### **1. Découverte de l'environnement**

Dans un premier temps, l'objectif va être de se familiariser avec les attributs et méthodes de Gym pour retrouver les propriétés de l'environnement Taxi-v3 énoncées dans le cours. Ces informations permettront de faciliter par la suite la compréhension et l'implémentation de l'algorithme du Q-Learning.

**Remarque :** certaines questions ne demandent pas nécessairement d'écrire du code. Dans ce cas, la cellule est au format Markdown.

Pour obtenir une description de l'environnement, utiliser **help(env.unwrapped)**

<font color=orange>**Question 1** : Afficher l'environnement de Taxi.</font>

<font color=orange>**Question 2** : Quelles récompenses l'agent peut-il obtenir pour une action ? Quelle est la récompense totale minimale à la fin d'une partie ? maximale si l'agent doit juste déposer le passager ?</font>

Pour information, une partie se termine au bout de 200 étapes si le passager n'a pas été déposé.

<font color=orange>**Question 3** : Retrouver les informations sur l'espace d'action présentées en cours (discret, 6 actions et associer à chaque valeur l'action réelle associée). </font>

<font color=orange>**Question 4** : Retrouver les informations sur l'espace d'observation présentées en cours (discret, 500 états).</font>

Pour rappel, il y a 500 états car :
- le taxi peut se retrouver à 25 positions différentes (grille de taille 5x5)
- 4 positions (R,G,Y,B) indiquant les positions possibles de départ et d'arrivée du passager
- 5 positions indiquant les positions possibles du passager avec 4 positions pour le départ et l'arrivée (R,G,B,Y) et une position quand le passager est dans le taxi

Avant d'implémenter l'algorithme du Q-Learning, nous allons commencer par faire jouer notre agent de manière aléatoire et observer la récompense par partie. Peut-être que sans entraînement, notre agent est capable de prendre puis de déposer le passager à sa destination...

### **2. Agent Random**

<font color=orange>**Question 5** : Générer un agent jouant 2 parties de Taxi de manière random et afficher en sortie la récompense par épisode.</font>  

Exemple de sortie :  
Reward of the episode 0 :  
Reward of the episode 1 :  

Indice : pour choisir une action aléatoire, utilisez **env.action_space.sample()**

**Remarque :** pour le problème du Taxi uniquement, entourez la méthode env.render() de la sorte :
- clear_output(wait = True)  (attention : cette commande vide la cellule du notebook. Si vous affichez les récompenses en temps réel, elles disparaîtront à la fin de l'action suivante.)
- env.render()
- time.sleep(0.1) (permet de faire jouer plus lentement notre agent)

Il est très probable que votre agent n'arrive jamais à déposer à prendre votre passager et à le déposer à sa destination. Pour y remédier, passons au Q-Learning.

### **3. Q-Learning**

Avec le Q-Learning, votre agent devrait apprendre au fur et à mesure des actions qu'il effectue.

Pour rappel, le Q-Learning permet de mesurer la qualité d'une combinaison état-action en termes de récompenses. Il le fait à l'aide d'une Q-Table qui est mise à jour après chaque action avec sa ligne correspondant à l'état et sa colonne à l'action. Un épisode se termine après la réalisation d'un ensemble d'actions. A la fin de l'entraînement, la Q-Table suggère la politique idéale.  
  
La mise à jour de la Q-Table se fait avec l'équation de Bellman : 

![](./images/bellman.png)

Nous allons commencer par définir la Q-Table. Les espaces d'observation et d'action étant discret, celle-ci est de la forme Q(nbEtat, nbAction). Pour l'initialisation de la Q-Table, vous pouvez l'initialiser à 0 ou à tout autre valeur car l'algorithme du Q-Learning garantit la convergence.

<font color=orange>**Question 6** : Définir la Q-Table.</font>

Indice : np.zeros
  
Exemple : np.zeros((2, 1))  
array([[ 0.],  
       [ 0.]])

<font color=orange>**Question 7** : Compléter les parties manquantes de l'algorithme du Q-Learning.</font>

In [ ]:
def QLearning(env, Q, alpha_apprentissage, gamma_actualisation, epsilon, min_eps, episodes):

    # Initialisation de variables pour suivre les rewards
    reward_list = []  
    avg_reward_list = []

    # Calcul d'un facteur de réduction pour epsilon. Pour rappel, epsilon est utilisé lors du choix de l'action pour trouver un compromis entre l'exploration de nouveaux états ou l'exploitation des connaissances déjà acquises.
    # Ce facteur de réduction permet de faire décroître de manière linéaire epsilon au cours du temps afin d'accorder plus de poids aux connaissances acquises dans la Q-Table.
    reduction = (epsilon - min_eps)/episodes

    for episode in range(episodes):

        # Initialisation des paramètres
        tot_reward, reward = 0, 0
        
        # TO-DO : Initialiser l'environnement, récupérer l'état initial et boucler tant que la partie n'est pas terminée

   
        while False:

            # TO-DO : Afficher l'environnement lors des 10 dernières parties de l'entraînement
            # if 


            # A compléter : déterminer la prochaine action : Epsilon Greedy Policy (trouver un compromis entre exploration et exploitation de la Q-Table)
            if np.random.random() < 1 - epsilon:
                # TO-DO : action = meilleure action de la Q-Table pour l'état en cours. Utilisez la méthode np.argmax.

            else:
                # TO-DO : action = random. On découvre autre chose
                
                
            # TO-DO : Effectuer l'action choisie et récupérer les informations

            # TO-DO : Mettre à jour la Q-Table en utilisant l'équation de Bellman
               
            # TO-DO : Mettre à jour notre récompense totale et l'état
            # tot_reward +=      
            # state =       
            
        
        # Réduire l'exploration lors du Epsilon Greedy Policy en réduisant l'epsilon pour profiter davantage des connaissances acquises dans la Q-Table.
        # Dans notre cas, on fait réduire epsilon de manière linéaire jusqu'à un minimum spécifié (min_eps) par rapport au nombre total d'épisodes.
        if epsilon > min_eps:
            epsilon -= reduction

        reward_list.append(tot_reward)

        if (episode+1) % 100 == 0:
            avg_reward = np.mean(reward_list)
            avg_reward_list.append(avg_reward)
            reward_list = []
            print('Episode {} Average Reward: {}'.format(episode+1, avg_reward))

    env.close()
        
    return avg_reward_list, Q

<font color=orange>**Question 8** : Exécuter l'algorithme précédent avec la cellule suivante. Un graphique "rewards.jpg" devrait avoir été créé. Celui-ci présente l'évolution de la récompense au fil de l'entrainement. Tester l'algorithme de Q-Learning pour différentes valeurs du facteur d'apprentissage et du facteur d'actualisation. Pour rappel : </font>

<img src="./images/apprentissage.PNG" width="500"/>

<img src="./images/actualisation.PNG" width="500"/>

In [ ]:
# Lancer l'apprentissage
rewards, qtable = QLearning(env, Q, alpha_apprentissage=0.2, gamma_actualisation=0.9, epsilon=0.8, min_eps=0.01, episodes=5000)

# Générer le graphique des rewards au cours des épisodes (rewards.jpg)
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.savefig('TaxiRewards.jpg')     
plt.close()  

Normalement votre agent est capable de résoudre aisément le problème, félicitations !  
  
A présent, pour montrer que les environnements standardisés sont un atout de Gym, nous allons changer d'environnement et adapter légèrement notre algorithme de Q-Learning à un nouveau problème plus visuel (et un peu plus complexe).

## B. MountainCar

Le problème : 
- Une voiture est placée sur une piste unidimensionnelle entre deux sommets.
- Objectif : gravir le sommet de droite
- Problème : le moteur de la voiture n’est pas assez puissant pour gravir le sommet d’un seul coup

![](./images/mountaincar.jpg)

In [ ]:
# TO-DO : Configuration de l'environnement appelé MountainCar-v0

### **1. Découverte de l'environnement**

Dans un premier l'objectif va être de comprendre l'environnement de MountainCar afin de faciliter par la suite la modification de l'implémentation de l'algorithme du Q-Learning.

<font color=orange>**Question 1** : Quand une partie (= épisode) se termine-t-elle d'un point de vue théorique ? (2 cas possibles)</font>

<font color=orange>**Question 2** : Quelle est la récompense totale minimale à la fin d'une partie (pire partie) ? maximale (partie parfaite) ?</font>

<font color=orange>**Question 3** : L'espace des actions de l'environnement est-il discret ou continu ? Décrire les différentes actions (mathématiquement) et l'action réelle associée à chaque valeur.</font>

<font color=orange>**Question 4** : L'espace des observations de l'environnement est-il discret ou continu ? S'il est discret décrire le nombre d'observations et leur signification sinon décrire les bornes inférieures et supérieures. S'il est continu, identifier également à quoi correspondent les composantes des observations.</font>

<font color=orange>**Question 5** : Dans le cadre du Q-Learning, l'espace d'observation pose problème lors de la création d'une Q-Table, pourquoi ?</font>

Avant d'implémenter l'algorithme du Q-Learning, nous allons commencer par faire jouer notre agent de manière aléatoire et observer la récompense par partie. Peut-être que sans entraînement, notre agent est capable de gravir le sommet...

### **2. Agent Random**

Afin de voir en pratique une première fois l'environnement de MountainCar, réalisons de nouveaun agent random.

<font color=orange>**Question 6** : Générer un agent jouant 3 parties au MountainCar de manière random et afficher en sortie la récompense par épisode.</font>  

Exemple de sortie :  
Reward of the episode 0 : -200.0  
Reward of the episode 1 : -200.0  
Reward of the episode 2 : -200.0  

Normalement, votre agent ne devrait jamais réussir à remporter une partie. Pour y remédier, passons au Q-Learning.

### **3. Q-Learning**

Par rapport à la version du Taxi, il va falloir adapter légèrement notre algorithme pour l'adapter à l'espace d'observation continu de MountainCar.  

Afin de générer la Q-Table, nous allons devoir discrétiser l'espace d'observation car nous avons vu que ce dernier est continu. La Q-Table correspondant à l'ensemble des paires états-actions possibles, nous devons déterminer le nombre d'états possibles pour créer la table aux bonnes dimensions. Notre espace d'observation ayant 2 composantes vectorielles, notre Q-Table sera de la forme Q(s1, s2, a). Comment connaître la taille de s1 et s2 ?  
  
L'espace d'observation ayant pour limite inférieure (min1 = -1.2, min2 = -0.07) et pour limite supérieure (max1 = 0.6, max2 = 0.07), une manière simple de discrétiser consiste à arrondir le premier élément du vector d'état à 0.1 près et le second élément à 0.01.  
Exemple : 
- Pour la première composante dont les limites sont (min = -1.2 et max = 0.6), les valeurs possibles après discrétisation sont les suivantes : -1.2, -1.1, -1.0, ..., 0.5, 0.6. Nous obtenons ainsi N valeurs (à déterminer à la question suivante).

**Remarque :**  
Une autre alternative consiste à utiliser le Deep Q-Learning. Cette méthode consiste à approximer l'algorithme de la Q-table avec un réseau de neurones. Cette solution est pertinente dans le cas de problèmes complexes notamment avec des espaces continus ou lorsque la taille de la Q-Table est très conséquente. Le Deep Q-Learning ne sera pas vu dans le cadre de ce cours mais constitue une bonne ouverture pour ceux voulant en savoir plus, cette solution étant très utilisée. Par ailleurs, pour un problème aussi simple que le MountainCar, on peut éviter de sortir les grands moyens et obtenir des résultats équivalents.

<img src="./images/dqn.png" width="1000"/>

<font color=orange>**Question 7** : Déterminer pour chacune des composantes de l'espace d'observation, le nombre de valeurs possibles après discrétisation.</font>

Indices : 
- Solution 1 : env.observation_space.high, env.observation_space.low, np.array([10,100]), np.round(..., decimals=0).astype(int), +1.  
- Solution 2 : np.arange(). Exemple : np.arange(0, 1, 0.1).size retourne 10
- Indice communs aux 2 solutions : ne pas oublier d'inclure les bornes  
En cas de difficultés, vous pouvez déterminer ces 2 valeurs à la main.

<font color=orange>**Question 8** : Définir la Q-Table (format 3D).</font>

Une fois le nombre de valeurs possibles par composante de l'espace d'observation obtenu, il est nécessaire de faire correspondre les valeurs de l'espace d'observation aux indices de notre table.  
Exemple :   
- Pour la première composante :  
![](./images/mapping_discretization.png)

**Remarque** : dans la première colonne, pleins d'autres valeurs décimales comprises entre -1.2 et 0.6 sont possibles ! Dans ce cas, il faut arrondir.  
Exemple : pour 0.02, on obtiendrait 12

<font color=orange>**Question 9** : Déterminer pour un état donné sa correspondance dans la Q-Table après discrétisation. Pour essayer votre code, vous pourrez prendre comme état **state = env.reset()**. state est dans ce cas de la forme (X, 0) avec X variable. En sortie la deuxième composante de state après discrétisation devrait être 7.</font>

Exemple :
- state = env.reset() peut retourner [-0.40525229 0.]. La première composante est variable et la seconde est toujours nulle.
- votre script devra retourner [8 7] dans ce cas. Ici -0.405... correespond à l'état 8 de la position. La vitesse (2ème composante) sera toujours 7 car initialement la vitesse est nulle.

Ce morceau de code sera à réutiliser dans l'algorithme du Q-Learning.  
  
Indice : np.array([10, 100]), np.round(..., 0).astype(int)

<font color=orange>**Question 10** : Compléter l'algorithme du Q-Learning adapté au problème de MountainCar.</font>

Indice :  
- Réutiliser l'algorithme du Taxi
- A chaque fois qu'un état est obtenu, le discrétiser
- Pour accéder à un élément d'un tableau 3D : Q[s1, s2, a]
- Ne pas réutiliser clear_output(wait = True) et time.sleep()

In [ ]:
def QLearning(env, Q, alpha_apprentissage, gamma_actualisation, epsilon, min_eps, episodes):

    
        
    return avg_reward_list, Q

<font color=orange>**Question 11** : Exécuter l'algorithme précédent avec la cellule suivante. Un graphique "rewards.jpg" devrait avoir été créé. Celui-ci présente l'évolution de la récompense au fil de l'entrainement.</font>

Cet apprentissage étant plus long que celui de Taxi, on peut sauvegarder la Q-Table. Ainsi si votre notebook plante ou pour une autre raison vous n'aurez pas à relancer l'apprentissage.

In [ ]:
# Lancer l'apprentissage
rewards, qtable = QLearning(env, Q, alpha_apprentissage=0.2, gamma_actualisation=0.9, epsilon=0.8, min_eps=0.01, episodes=5000)

qtable = qtable.reshape(285,3)
np.savetxt("Qtable.csv", qtable, delimiter=";")

In [ ]:
# Générer le graphique des rewards au cours des épisodes (rewards.jpg)
plt.plot(100*(np.arange(len(rewards)) + 1), rewards)
plt.xlabel('Episodes')
plt.ylabel('Average Reward')
plt.title('Average Reward vs Episodes')
plt.savefig('rewards.jpg')     
plt.close()  

Récupération de la Q-Table et restructuration en 3D.

In [ ]:
# Ouverture du fichier Qtable.csv pour récupérer la table apprise puis reshape en 3d
Q = np.loadtxt("Qtable.csv", delimiter=";")
Q = Q.reshape(19,15,3)

<font color=orange>**Question 12** : Utiliser la Q-Table pour résoudre le problème.</font>  

### Le TP touche à sa fin... Félicitations !  